# Домашнее задание 2

Это домашнее задание по материалам 5-8 занятий. Дедлайн сдачи - __23:59 4 мая__.

- Домашнее задание выполняется в этом же Jupyter Notebook.

- Перед отправкой файл нужно переименовать: __Фамилия_Имя__(без пробелов в начале и конце). Пример: __Иванов_Иван__.

- Домашнее задание присылается на адрес __OptimizationHomework@yandex.ru__. Тема письма: __ИППИ_Номер_задания__ (без пробелов в начале и конце). Для этого задания тема письма: __ИППИ_2__.

- Поместите решение каждой задачи/пункта после условия.

- Не забывайте добавлять необходимые пояснения и комментарии.

- Все технические артефакты должны быть удалены в финальной версии, которая будет отправлена на проверку. Под такими артефактами понимаются любые выводы ячеек, которые никак не прокомментированы в тексте, а также любые объемные/длинные технические выводы (даже если они прокомментированы в тексте).

- Запуск решения (Kernel -> Restart & Run All) должен отработать все ячейки без ошибок.

- Максимальный балл за задание - 150. Для получения полного балла за домашнее задание необходимо набрать 100 баллов.

Желаем вам успехов!

### Решение задачи распределенной оптимизации

Снова рассмотрим задачи минимизации эмпирического риска:
\begin{equation}
\min_{w \in \mathbb{R}^d} \frac{1}{n} \sum\limits_{i=1}^n \ell (g(w, x_i), y_i) + \frac{\lambda}{2} \| w\|^2_2,
\end{equation}
где $\ell: \mathbb{R} \times \mathbb{R} \to \mathbb{R}$ - функция потерь, $g : \mathbb{R}^d \times \mathbb{R}^x \to \mathbb{R}$ - модель, $w$ - параметры модели, $\{x_i, y_i\}_{i=1}^n$ - обучающая выборка из объектов $x_i \in \mathbb{R}^x$ и меток $y_i \in \mathbb{R}$.

Будем использовать линейную модель $g(w, x) = w^T x$ и логистическую/сигмоидную функцию потерь: $\ell(z,y) = \ln (1 + \exp(-yz))$ (__важно: $y$ должен принимать значения $-1$ или $1$__). Как мы уже знаем, полученная задача называется задачей логистической регрессии.

Рассматриваемую задачу можно переписать следующим образом:
\begin{equation}
\min_{w \in \mathbb{R}^d} f(w) := \frac{1}{s} \sum\limits_{j=1}^s f_j(w) := \frac{1}{s} \sum\limits_{j=1}^s \left[\frac{1}{b} \sum\limits_{i=1}^b l (g(w, x_{(j-1)b + i}), y_{(j-1)b + i}) + \frac{\lambda}{2} \| w\|^2_2\right],
\end{equation}
где $b$ - размер локальной выборки, $s$ - количество устройств, а $b s = n$ - общий размер обучающей выборки.

Градиент для $f_j$ можно записать следующим образом:
$$
\nabla f_j(w) = \frac{1}{b} \sum_{i=1}^b \frac{-y_{(j-1)b + i} x_{(j-1)b + i}}{1 + \exp(y_{(j-1)b + i} w^Tx_{(j-1)b + i})} + \lambda w.
$$
Константа Липшица градиента $\nabla f_j$ может быть оценена как $L_j = \frac{1}{4b} \sum\limits_{i=1}^b \| x_{(j-1)b + i} \|^2_2 + \lambda$.

Проведем небольшую подготовительную работу. К заданию, как и в прошлый раз, приложен датасет _mushrooms_. Используйте следующий код для создания матрицы $X$ и вектора $y$, которые будут отвечать за выборку $\{x_i, y_i\}_{i=1}^n$:

In [1]:
dataset = "mushrooms.txt" 
#the file must be in the same directory as notebook.

In [2]:
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]

Изменим вектор $y$ так, чтобы $y_i$ принимал значения $-1$ и $1$. Вы также можете провести дополнительную предобработку данных (с помощью изветсных вам методов), но это никак не оценивается дополнительно.

In [4]:
y = 2 * y - 3

Разделим данные на две части: обучающую и тестовую.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Для обучающей части $X_{train}$, $y_{train}$ оцените константу $L$. Выберите $\lambda$ так, чтобы $\lambda \approx L / 1000$.  Реализуйте в коде вычисление значения и градиента для нашей целевой функции ($X$, $y$, $\lambda$ должны быть заданы в качестве параметра, чтобы их можно было менять, а не только подставлять фиксированные $X_{train}$, $y_{train}$). Для реализации Вы можете использовать любую из библиотек: ``numpy``, `autograd``, ``pytorch`` или ``jax``.

In [ ]:
#ваше решение (Code и Markdown)

__Задача 0. (10 + 20  баллов)__ Поскольку мы имеем дело с распределенной средой, нам нужно уметь делить данные на части. Скорее всего, Вы уже делали это в прошлом задании, когда делили данные на батчи. 

In [ ]:
#ваше решение (Code и Markdown)

Но для получения более интересных результатов можно реализовать дополнительные возможности:

__a). (10 баллов)__ Реализуйте возможность разделить данные неравномерно/гетерогенно, например, дать некоторым устройствам преимущественно данные с метками, равными $1$, а другим с $- 1$. Вы можете использовать этот пример или придумать свой собственный способ неоднородного разделения данных. В любом случае опишите свой подход и реализуйте его. В идеале Вы должны иметь возможность варьировать степень неоднородности от "равномерно разделения" до "абсолютно разнородных локальных данных". 

In [ ]:
#ваше решение (Code и Markdown)

Распределенные вычисления можно смоделировать на одном устройстве. В этом домашнем задании будет достаточно, если сделать это самым тривиальным образом с помощью дополнительного цикла.

__б). (бонус 20 баллов)__ Но Вы можете подключить некоторые специализированные библиотеки для параллельных вычислений. Например, библиотеку ``ray``, но можно использовать и другие решения. Опишите, что вы собираетесь сделать. Мы советуем Вам сделать этот пункт, если Вы понимаете, что у Вас достаточно времени и ресурсов для организации параллельных/распределенных вычислений. 

In [ ]:
#ваше решение (Code и Markdown)

__Задача 1. (30 + 30 баллов)__ Эта часть задания связана с __локальными распределенными методами__.

__a). (20 баллов)__ Смоделируйте распределенную среду с $s = 10$ и равномерно распределенными данными. Реализуйте простой локальный метод FedAvg (Алгоритм 1 из [статьи](https://arxiv.org/pdf/1909.04746.pdf), в статье рассматривается SGD, но нам будет достаточно GD), где в качестве параметра можно выбрать количество локальных шагов $H$. Шаг метода возьмите равным $\frac{1}{L}$. Что такое $L$ здесь? Ответ на этот вопрос можно найти в упомянутой статье.

Варьируйте количество локальных шагов: $H = 1, 5, 10, 20$. Постройте графики сравнения для разных $H$: 1) значение критерия сходимости в зависимости от числа раундов общения, 2) точность предсказаний в зависимости от числа раундов общения. Сделайте вывод.

In [3]:
#ваше решение (Code и Markdown)

__б). (10 баллов)__ Давайте зафиксируем $s = 10$, $H = 10$ и будем менять степень неоднородности деления данных. Шаг метода все еще равен $\frac{1}{L}$ (учтите, что для каждой случая неоднородности лучше вычислять $L$ заново). Постройте графики сравнения для $3-5$ различных степеней неоднородности деления данных: 1) значение критерия сходимости в зависимости от числа раундов общения, 2) точность предсказаний в зависимости от числа раундов общения. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__в). (бонус 30 баллов)__ Надеемся, что в предыдущем пункте Вам удалось понять, что при сильной неоднородности локальных данных результаты Local GD становятся все хуже и хуже. Для решения этой проблемы был предложен метод SCAFFOLD. Реализуйте SCAFFOLD (Алгоритм 1 из [статьи](http://proceedings.mlr.press/v119/karimireddy20a/karimireddy20a.pdf), случайный выбор усстрйоств не нужен: в строке 4 $S = \{1, \ldots, N \}$, стохастические градиенты тоже не нужны: используйте полные локальные градиенты). Повторите пункты a)-б) для SCAFFOLD и сравните его с Local SGD.

In [4]:
#ваше решение (Code и Markdown)

__Задача 2. (60 баллов)__ Эта часть задания связана с методами __распределеной оптимизации со сжатием__.

__а). (10 баллов)__ Реализуйте операторы сжатия $\text{Rand}k$ и $\text{Top}k$, где $k$ - параметр. 

In [ ]:
#ваше решение (Code)

__б). (15 баллов)__ Смоделируйте распределенную среду с $s=10$ и равномерно распределенными данными. Реализуйте обычный распределенный градиентный спуск с операторами сжатия. Сжимайте информацию только от устройств к серверу. Используйте операторы сжатия $\text{Rand}1\%$, $\text{Rand}5\%$, $\text{Rand}10\%$, $\text{Rand}20\%$. Шаги метода равны $L^{-1} (\frac{2 \omega}{s} + 1)^{-1}$ (теоретические). Объясните, что такое $L$ и $\omega$ в данном случае. 

Постройте графики сравнения для разных операторов: 1) зависимости значения критерия сходимости от количества переданных координат с устройств на сервер; 2) зависимости точности предсказания от количества переданных координат с устройств на сервер. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__в). (10 баллов)__ Попробуйте подобрать шаг метода для каждого оператора сжатия из предыдущего пункта. Объясните, как Вы определили, что этот шаг "самый лучший" (здесь нет универсального рецепта, только Ваши мысли).

Постройте графики сравнения для различных операторов и настроенных для них шагов: 1) зависимости значения критерия сходимости от количества переданных координат с устройств на сервер; 2) зависимости точности предсказания от количества переданных координат с устройств на сервер. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__г). (10 баллов)__ Зафиксируем $s = 10$, $\text{Rand}10\%$ и будем менять степень неоднородности деления данных. Шаг равен $L^{-1} (\frac{2 \omega}{s} + 1)^{-1}$ (учтите, что для каждого случая неоднородности лучше вычислять $L$ заново). Постройте графики сравнения для $3-5$ различных степеней неоднородности: 1) зависимости значения критерия сходимости от количества переданных координат от устройств к серверу; 2) зависимости точности предсказания от количества переданных координат от устройств к серверу. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__д). (15 баллов)__ Теперь давайте использовать $s=10$, равномерно распределенные данные и $\text{Top}10\%$ в качестве операторов сжатия. Сжимайте информацию только от устройств к серверу. Используйте два метода: простой распределенный градиентный спуск с операторами сжатия и распределенный градиентный спуск с техникой компенсации ошибки (см. Алгоритм 1 из [статьи](https://arxiv.org/abs/2002.12410)). Для каждого метода выберите шаг, обеспечивающий "наилучшую" сходимость. Постройте графики сравнения для двух подходов без техникой компесации и с ней с "лучшими" шагами: 1) зависимости значения критерия сходимости от количества переданных координат с устройств на сервер; 2) зависимости точности предсказания от количества переданных координат с устройств на сервер. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)